# **Name: Md. Azharul Islam <br>
# ID: 181-35-2329**


In [1]:
#import all the libraries
import os
import string

import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

import torchvision
#from torchvision import models
from torchvision import transforms
from torchvision.transforms import ToTensor, Normalize

In [ ]:
#ord('C') - ord('A')

In [2]:
#Train dataset config 
# Two mappings have been made here. 
#The first is from charachter to integer and the second is from integer to charachter. So first I did integer mapping from charachter.
class ASLDatasetTrain(Dataset):
    char_to_int = {c: ord(c) - ord('A') for c in string.ascii_uppercase} #I have created a dictionary here.
    char_to_int['del'] = 26
    char_to_int['nothing'] = 27
    char_to_int['space'] = 28
    int_to_char = {value: key for key, value in char_to_int.items()}
    #asci_uppercase mainly return the all upper case alphabet and i take e key value. Suppose if the alphabet 'A' then the outcome is zero.
    #More explanation suppose if you run ord('A') then you get the A alphabet value but we need to sort out of all the values so 
    #what we've done here is we've got a key value for each alphabet.
    #Where the value of A is zero then the value of B continuously is 2 and thus I have taken 26 key values of 26 alphabets.
    #And thats the way to get the each key value - ord('B') - ord('A')
    #Since I have taken the key value of our 27 alphabets as 26 (0-25), 
    #on the other hand I had 3 extra classes and I have taken those 3 classes as 26, 27, 28 respectively.
    #And finally i convert to integer value to character value.
        
    def __init__(self, directory: str, transform=None, label_transform=None):
        #I have given some parameters here where I have set the directory which means I have input the directory in which I have the dataset. 
        #Then I did no transformation for the dataset and did the label transformation none.
        super().__init__()
        
        self.directory = directory #Since I have created separate functions for train and test, I have just given a directory here.
        self.transform = transform
        self.label_transform = label_transform
        
        self.x = None
        self.y = None
        # This portion I am setting the data as none and also setting the target as none.
        self._load_images()
        #And finally i call my helper function.
    
    def __getitem__(self, idx):
        x, y = torchvision.io.read_image(self.x[idx]).type(torch.float32), self.y[idx]
        
        if self.transform:
            x = self.transform(x)
        if self.label_transform:
            y = self.label_transform(y)
        
        return x, y
    
    def __len__(self):
        return len(self.y)
    
    def _load_images(self):
        self.x = []
        self.y = []
        
        for c in os.listdir(self.directory):
            class_name = c
            class_dir = os.path.join(self.directory, class_name)
            for img in os.listdir(class_dir):
                self.x.append(os.path.join(class_dir, img))
                self.y.append(self.char_to_int[class_name])
                
        self.y = torch.tensor(self.y, dtype=torch.int64)
    #I set the directory which is train or test, whatever it is, I will run a loop in all the folders. 
    #I am running a list of all the folders that I have set in self.directory. We can see from the dataset that the name of the directory is the class name. 
    #Then the class name will be (C). Then we enter the class directory and return all the files / folders in that directory.
    #At the end of in the load image function, 
    #all image paths are being stored in self.x and all image labels are being stored in self.y.
    @staticmethod
    def get_classname(idx: int) -> str:
        return ASLDatasetTrain.int_to_char[idx] # This function mainly work, suppose if you take (0) id then it returns alphabet (A) mainly this function return integer to charachter value.

In [3]:
#Test dataset config
class ASLDatasetTest(Dataset):
    char_to_int = {c: ord(c) - ord('A') for c in string.ascii_uppercase}
    char_to_int['del'] = 26
    char_to_int['nothing'] = 27
    char_to_int['space'] = 28
    int_to_char = {value: key for key, value in char_to_int.items()}
        
    def __init__(self, directory: str, transform=None, label_transform=None):
        super().__init__()
        
        self.directory = directory
        self.transform = transform
        self.label_transform = label_transform
        
        self.x = None
        self.y = None
        
        self._load_images()
    
    def __getitem__(self, idx):
        x, y = torchvision.io.read_image(self.x[idx]).type(torch.float32), self.y[idx]
        
        if self.transform:
            x = self.transform(x)
        if self.label_transform:
            y = self.label_transform(y)
        
        return x, y
    
    def __len__(self):
        return len(self.y)
    
    def _load_images(self):
        self.x = []
        self.y = []
        
        for img in os.listdir(self.directory):
            class_name = img [:1]
            if 'space' in img:
                class_name = 'space'
            elif 'nothing' in img:
                class_name = 'nothing'
            elif 'del' in img:
                class_name = 'del'    
            self.x.append(os.path.join(self.directory, img))
            self.y.append(self.char_to_int[class_name])
                
        self.y = torch.tensor(self.y, dtype=torch.int64)
    
    @staticmethod
    def get_classname(idx: int) -> str:
        return ASLDatasetTest.int_to_char[idx]
    #The DatasetTest function will work the same as the DatasetTrain function, but the load image function will change a bit because my test dataset was a little different. 
    #The test dataset contained all the images in a folder, so I used a condition in the load image function. 
    #In that condition if the name of the image is 'space', 'nothing' or 'dell' then the class name will be 'space', 'nothing' or 'dell'.

In [4]:
#Transformation
ts = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(224),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])
#Here's what I think, 
#resize has taken more than the required size of an image here (256) and taken randomly (224).
#And also i think that are ratio issues thats why use the size 256.
#Since I am using AlexNet, this value works best to normalize so I am using it.

In [5]:
#All the data store in train and test
train = ASLDatasetTrain('../input/fingerdataset/dataset/train', transform=ts)
test = ASLDatasetTest('../input/fingerdataset/dataset/test', transform=ts)
#Now we will load the data as my test dataset is different so I have taken two functions to load.

In [6]:
#Train and test data print
print(len(train))
print(len(test))

812
203


In [7]:
#test path
test.x
#to check the each data 

['../input/fingerdataset/dataset/test/O2112.jpg',
 '../input/fingerdataset/dataset/test/L2080.jpg',
 '../input/fingerdataset/dataset/test/G331.jpg',
 '../input/fingerdataset/dataset/test/T1452.jpg',
 '../input/fingerdataset/dataset/test/W575.jpg',
 '../input/fingerdataset/dataset/test/Q2011.jpg',
 '../input/fingerdataset/dataset/test/I2044.jpg',
 '../input/fingerdataset/dataset/test/V1796.jpg',
 '../input/fingerdataset/dataset/test/F1236.jpg',
 '../input/fingerdataset/dataset/test/F1804.jpg',
 '../input/fingerdataset/dataset/test/R1048.jpg',
 '../input/fingerdataset/dataset/test/H1828.jpg',
 '../input/fingerdataset/dataset/test/A644.jpg',
 '../input/fingerdataset/dataset/test/W1143.jpg',
 '../input/fingerdataset/dataset/test/C691.jpg',
 '../input/fingerdataset/dataset/test/nothing324.jpg',
 '../input/fingerdataset/dataset/test/W1303.jpg',
 '../input/fingerdataset/dataset/test/X2276.jpg',
 '../input/fingerdataset/dataset/test/Y2124.jpg',
 '../input/fingerdataset/dataset/test/Q8.jpg',
 '

In [8]:
#train path
train.x
#to check the each data

['../input/fingerdataset/dataset/train/N/N1.jpg',
 '../input/fingerdataset/dataset/train/N/N2059.jpg',
 '../input/fingerdataset/dataset/train/N/N2698.jpg',
 '../input/fingerdataset/dataset/train/N/N531.jpg',
 '../input/fingerdataset/dataset/train/N/N529.jpg',
 '../input/fingerdataset/dataset/train/N/N1465.jpg',
 '../input/fingerdataset/dataset/train/N/N1032.jpg',
 '../input/fingerdataset/dataset/train/N/N2155.jpg',
 '../input/fingerdataset/dataset/train/N/N2060.jpg',
 '../input/fingerdataset/dataset/train/N/N2700.jpg',
 '../input/fingerdataset/dataset/train/N/N2057.jpg',
 '../input/fingerdataset/dataset/train/N/N530.jpg',
 '../input/fingerdataset/dataset/train/N/N1468.jpg',
 '../input/fingerdataset/dataset/train/N/N2154.jpg',
 '../input/fingerdataset/dataset/train/N/N1466.jpg',
 '../input/fingerdataset/dataset/train/N/N1467.jpg',
 '../input/fingerdataset/dataset/train/N/N3.jpg',
 '../input/fingerdataset/dataset/train/N/N2153.jpg',
 '../input/fingerdataset/dataset/train/N/N2699.jpg',
 '

In [9]:
#sampler
train_sampler = SubsetRandomSampler(np.arange(len(train)))
test_sampler = SubsetRandomSampler(np.arange(len(test)))

In [10]:
#train and test loader using sampler
train_loader = DataLoader(train, 32, sampler=train_sampler)
test_loader = DataLoader(test, 32, sampler=test_sampler)
#we create data loader with 32-32

In [11]:
for x, y in train_loader:
    print(x.shape)
    print(y.shape)
#To check the shape of the train loader
#Each mini-batch has 32 images and 3 color channels and dimension 224 * 224.

torch.Size([32, 3, 224, 224])
torch.Size([32])
torch.Size([32, 3, 224, 224])
torch.Size([32])
torch.Size([32, 3, 224, 224])
torch.Size([32])
torch.Size([32, 3, 224, 224])
torch.Size([32])
torch.Size([32, 3, 224, 224])
torch.Size([32])
torch.Size([32, 3, 224, 224])
torch.Size([32])
torch.Size([32, 3, 224, 224])
torch.Size([32])
torch.Size([32, 3, 224, 224])
torch.Size([32])
torch.Size([32, 3, 224, 224])
torch.Size([32])
torch.Size([32, 3, 224, 224])
torch.Size([32])
torch.Size([32, 3, 224, 224])
torch.Size([32])
torch.Size([32, 3, 224, 224])
torch.Size([32])
torch.Size([32, 3, 224, 224])
torch.Size([32])
torch.Size([32, 3, 224, 224])
torch.Size([32])
torch.Size([32, 3, 224, 224])
torch.Size([32])
torch.Size([32, 3, 224, 224])
torch.Size([32])
torch.Size([32, 3, 224, 224])
torch.Size([32])
torch.Size([32, 3, 224, 224])
torch.Size([32])
torch.Size([32, 3, 224, 224])
torch.Size([32])
torch.Size([32, 3, 224, 224])
torch.Size([32])
torch.Size([32, 3, 224, 224])
torch.Size([32])
torch.Size([3

# **Alexnet**

In [12]:
class AlexNet(nn.Module):
    def __init__(self, num_classes: int = 1000) -> None:
        super(AlexNet, self).__init__()
        
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [13]:
model = AlexNet(29)
#Initially there are 1000 classes here, 
#but since I have 29 classes here, I will define them in parameters.

In [14]:
#using adam optimizer
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.0001)


In [15]:
#print minibatch of the train data
print(len(train_loader.dataset))
print(len(train_loader))
#to check total data and mini-batch

812
26


In [16]:
epochs = 20

for e in range(epochs):
    running_loss = 0.0
    
    for i, (imgs, labels) in enumerate(train_loader):
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if i % 13 == 12:
            print('[Epoch %d, Step %5d] loss: %.3f' %
                  (e + 1, i + 1, running_loss / 13))
            running_loss = 0.0
#I want the result after 13 mini-batch so I have set 13 in condition.

[Epoch 1, Step    13] loss: 4.185
[Epoch 1, Step    26] loss: 3.405
[Epoch 2, Step    13] loss: 3.363
[Epoch 2, Step    26] loss: 3.352
[Epoch 3, Step    13] loss: 3.300
[Epoch 3, Step    26] loss: 3.281
[Epoch 4, Step    13] loss: 3.110
[Epoch 4, Step    26] loss: 2.895
[Epoch 5, Step    13] loss: 2.712
[Epoch 5, Step    26] loss: 2.484
[Epoch 6, Step    13] loss: 2.284
[Epoch 6, Step    26] loss: 1.924
[Epoch 7, Step    13] loss: 1.689
[Epoch 7, Step    26] loss: 1.486
[Epoch 8, Step    13] loss: 1.268
[Epoch 8, Step    26] loss: 1.262
[Epoch 9, Step    13] loss: 1.052
[Epoch 9, Step    26] loss: 1.008
[Epoch 10, Step    13] loss: 0.927
[Epoch 10, Step    26] loss: 0.809
[Epoch 11, Step    13] loss: 0.683
[Epoch 11, Step    26] loss: 0.609
[Epoch 12, Step    13] loss: 0.538
[Epoch 12, Step    26] loss: 0.417
[Epoch 13, Step    13] loss: 0.334
[Epoch 13, Step    26] loss: 0.414
[Epoch 14, Step    13] loss: 0.254
[Epoch 14, Step    26] loss: 0.296
[Epoch 15, Step    13] loss: 0.286
[Ep

In [17]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \nAccuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [18]:
test(test_loader, model, nn.CrossEntropyLoss())

Test Error: 
Accuracy: 50.2%, Avg loss: 3.152189 



In [19]:
#each class wise accuracy
class_correct = list(0. for i in range(29))
class_total = list(0. for i in range(29))

with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(len(labels)):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(29):
    print('Accuracy of %5s : %2d %%' % (
        ASLDatasetTest.int_to_char[i], 100 * class_correct[i] / class_total[i]))

Accuracy of     A : 42 %
Accuracy of     B : 71 %
Accuracy of     C : 57 %
Accuracy of     D : 57 %
Accuracy of     E : 42 %
Accuracy of     F : 71 %
Accuracy of     G : 57 %
Accuracy of     H : 42 %
Accuracy of     I : 28 %
Accuracy of     J : 28 %
Accuracy of     K : 71 %
Accuracy of     L : 42 %
Accuracy of     M : 57 %
Accuracy of     N : 57 %
Accuracy of     O : 57 %
Accuracy of     P : 14 %
Accuracy of     Q : 42 %
Accuracy of     R : 42 %
Accuracy of     S : 42 %
Accuracy of     T : 42 %
Accuracy of     U : 14 %
Accuracy of     V : 42 %
Accuracy of     W : 42 %
Accuracy of     X : 42 %
Accuracy of     Y : 28 %
Accuracy of     Z : 42 %
Accuracy of   del : 57 %
Accuracy of nothing : 71 %
Accuracy of space : 57 %


# **Pre-trained AlexNet**

In [20]:
from torchvision import models

**Issue on kaggle anyone trying to download the model will fail if the internet option is off at the bottom right.**

In [21]:
model = models.alexnet(pretrained=True)

Downloading: "https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-4df8aa71.pth


  0%|          | 0.00/233M [00:00<?, ?B/s]

In [22]:
for param in model.parameters():
    param.requires_grad = False

In [23]:
print(model)
#To check the model

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [24]:

new_clf = nn.Sequential(
    nn.Dropout(p=0.5, inplace=False),
    nn.Linear(in_features=9216, out_features=4096, bias=True),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.5, inplace=False),
    nn.Linear(in_features=4096, out_features=4096, bias=True),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=4096, out_features=1000, bias=True),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=1000, out_features=29, bias=True),
)
#Initially they define 1000 feature/class but i set the manually features 29

In [25]:
model.classifier = new_clf
#I have replaced the new classifier I created in the previous classifier.

In [26]:
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.0001)

In [27]:
epochs = 20

for e in range(epochs):
    running_loss = 0.0
    for i, (imgs, labels) in enumerate(train_loader):
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if i % 13 == 12:
            print('[Epoch %d, Step %5d] loss: %.3f' %
                  (e + 1, i + 1, running_loss / 13))
            running_loss = 0.0

[Epoch 1, Step    13] loss: 29.168
[Epoch 1, Step    26] loss: 11.136
[Epoch 2, Step    13] loss: 5.362
[Epoch 2, Step    26] loss: 3.957
[Epoch 3, Step    13] loss: 3.405
[Epoch 3, Step    26] loss: 3.092
[Epoch 4, Step    13] loss: 2.668
[Epoch 4, Step    26] loss: 2.624
[Epoch 5, Step    13] loss: 2.614
[Epoch 5, Step    26] loss: 2.239
[Epoch 6, Step    13] loss: 2.247
[Epoch 6, Step    26] loss: 2.286
[Epoch 7, Step    13] loss: 2.168
[Epoch 7, Step    26] loss: 1.914
[Epoch 8, Step    13] loss: 1.909
[Epoch 8, Step    26] loss: 1.753
[Epoch 9, Step    13] loss: 1.755
[Epoch 9, Step    26] loss: 1.789
[Epoch 10, Step    13] loss: 1.429
[Epoch 10, Step    26] loss: 1.513
[Epoch 11, Step    13] loss: 1.409
[Epoch 11, Step    26] loss: 1.430
[Epoch 12, Step    13] loss: 1.251
[Epoch 12, Step    26] loss: 1.321
[Epoch 13, Step    13] loss: 1.237
[Epoch 13, Step    26] loss: 1.223
[Epoch 14, Step    13] loss: 1.188
[Epoch 14, Step    26] loss: 0.977
[Epoch 15, Step    13] loss: 1.176
[

In [28]:
test(test_loader, model, nn.CrossEntropyLoss())

Test Error: 
Accuracy: 48.3%, Avg loss: 2.736922 



# ResNet101

In [29]:
#Used resnet 101 instead of resnet 152
model = models.resnet101(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet101-5d3b4d8f.pth" to /root/.cache/torch/hub/checkpoints/resnet101-5d3b4d8f.pth


  0%|          | 0.00/170M [00:00<?, ?B/s]

In [30]:
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [31]:
#create new fully connected layer which is the initially features 1000 but i need 29 features so i set this value.
new_fc = torch.nn.Sequential(
    nn.Linear(in_features=2048, out_features=1000, bias=True),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=1000, out_features=29, bias=True),
)

In [32]:
#As usal replace new fully connected layer
model.fc = new_fc

In [33]:
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.0001)

In [34]:
epochs = 5

for e in range(epochs):
    running_loss = 0.0
    for i, (imgs, labels) in enumerate(train_loader):
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if i % 13 == 12:
            print('[Epoch %d, Step %5d] loss: %.3f' %
                  (e + 1, i + 1, running_loss / 12))
            running_loss = 0.0
#As usal i want the result after 13 mini-batch so I have set 13 in condition and set 5 iteration.

[Epoch 1, Step    13] loss: 3.439
[Epoch 1, Step    26] loss: 2.664
[Epoch 2, Step    13] loss: 1.399
[Epoch 2, Step    26] loss: 0.551
[Epoch 3, Step    13] loss: 0.198
[Epoch 3, Step    26] loss: 0.105
[Epoch 4, Step    13] loss: 0.059
[Epoch 4, Step    26] loss: 0.063
[Epoch 5, Step    13] loss: 0.055
[Epoch 5, Step    26] loss: 0.069


In [35]:
test(test_loader, model, nn.CrossEntropyLoss())

Test Error: 
Accuracy: 93.1%, Avg loss: 0.234341 

